In [7]:
import cv2
import numpy as np
from time import *
# imagePath = 'C:/Users/Nolan/Documents/Python Scripts/npm/analysis/data/calibration 20x.png'
# imageName = imagePath.split(r'/')[-1].split('.')[0]
# image = cv2.imread(imagePath)

# if image is None:
#     raise ValueError("Could not load image. Check the file path.")

def scaleBar(image, scaleFactor=6.9, scaleLength = 30, scaleUnit = 'um', barHeight = 10, border = 1, divisions = 30, fontScale = 1, thickness = 1, posX = 20, posY = 30, font=cv2.FONT_HERSHEY_DUPLEX):
    scaleLengthPixels = int(scaleLength*scaleFactor)
    shape = image.shape
    if len(shape) == 2:  # Grayscale image
        height, width = shape
        channels = 1  # Grayscale has one channel
    elif len(shape) == 3:  # Color image
        height, width, channels = shape
    else:
        raise ValueError("Unexpected image shape.")
    msg = f'{scaleLength} {scaleUnit}'
    scaleBarStart = (posX+border, height - posY)
    scaleBarEnd = (scaleBarStart[0] + scaleLengthPixels, scaleBarStart[1]-barHeight)
    
    def rectBorder(start, end, borderL, borderR, borderU, borderD): # return border positions for any given rect
        newStart = (start[0]-borderL, start[1]-borderU)
        newEnd = (end[0] + borderR, end[1]+borderD)
        return [newStart, newEnd]

    (w, h), b = cv2.getTextSize(str(msg), font, fontScale, thickness)
    textPosition = (scaleBarStart[0], scaleBarStart[1] - h - b - barHeight - 5)  # Adjust position as needed
    
    def textBackground(text, fontScale, thickness, pos):
        (textWidth, textHeight), baseline = cv2.getTextSize(str(text), font, fontScale, thickness)
        rect = [(pos[0], pos[1]), (pos[0] + textWidth, pos[1] + textHeight+baseline)]
        textPos = (pos[0], pos[1] + textHeight + baseline // 2)
        
        return (rect, textPos)

    rect, textPos = textBackground(msg, fontScale, thickness, textPosition)
    cv2.rectangle(image, rect[0], rect[1], color=(255, 255, 255), thickness=-1) # background

    def evenDivision(n, f):
        # Step 1: Calculate the step size
        f=f+1
        if f > 1:
            stepSize = n / (f - 1)
        else:
            return np.array([0])  # If f is 1, return a single value
        
        # Step 2: Create evenly spaced values using rounding
        distributedArray = np.round(np.linspace(0, n, f)).astype(int)
        
        # Ensure that the last element is exactly n
        distributedArray[-1] = n
        
        divisions=np.insert(np.diff(distributedArray), 0, 0)
    
        return divisions
    
    dist = evenDivision(scaleLengthPixels, divisions)
    for i in range(0, len(dist)): # black/white divisions
        if i == len(dist)-1:
            break
        start = (scaleBarStart[0]+np.sum(dist[0:i+1]),scaleBarStart[1])
        end = (start[0]+dist[i+1],scaleBarEnd[1])
        if i % 2 == 0:
            cv2.rectangle(image, start, end, color=(0, 0, 0), thickness=-1)
        else:
            cv2.rectangle(image, start, end, color=(255, 255, 255), thickness=-1)
    
    cv2.rectangle(image, scaleBarStart, scaleBarEnd, color=(100, 100, 100), thickness=border) # scale bar border
    
    cv2.putText(image, msg, textPos, font, # text
                fontScale=fontScale, color=(0, 0, 0), thickness=thickness, lineType=cv2.LINE_AA)

def textBackground(text, fontScale, thickness, pos):
    (textWidth, textHeight), baseline = cv2.getTextSize(str(text), cv2.FONT_HERSHEY_SIMPLEX, fontScale, thickness)
    rect = [(pos[0], pos[1]), (pos[0] + textWidth, pos[1] + textHeight+baseline)]
    textPos = (pos[0], pos[1] + textHeight + baseline // 2)

    return (rect, textPos)

In [48]:
videoName = "D:/NPM/09-19-2024/DC/5V 30 sec.avi"
cap = cv2.VideoCapture(videoName)
ret, frame = cap.read()
# get the video frame height and width
height, width, channel = frame.shape
            
frame_count = 0

backgroundSubtractor = cv2.createBackgroundSubtractorMOG2(history=450, varThreshold=16 + 48 * 0.3)
#cap.set(cv2.CAP_PROP_FPS, 30)
frameCount=0
frames=[]
CF=12
temp=0
evenFrame=None
oddFrame=None
while (cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        fontScale = 1.5
        thickness = 2
        timer = cv2.getTickCount()
        frame_count += 1
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame = backgroundSubtractor.apply(frame)
        frameCount+=1

        

        _, frame = cv2.threshold(frame, 50, 255, cv2.THRESH_BINARY)
        frame = cv2.GaussianBlur(frame, (13,13), 0)
        frame = cv2.Canny(frame, 50, 150)
        if frameCount % 2 == 0:
            evenFrame=frame.copy()
            temp=cap.get(cv2.CAP_PROP_POS_FRAMES)
        elif frameCount > temp or frameCount == 1: # make sure that it takes the difference from a frame and a previous frame, not sure if necessary
            oddFrame=frame.copy()
        if evenFrame is not None and oddFrame is not None:
            frames.append(cv2.absdiff(oddFrame, evenFrame)) # find difference between current 2 most recent frames

        if len(frames) > CF:
            frames.pop(0)
        if len(frames) > CF + 1:
            self.printNewLine(f'Cached frames "{len(frames)}" exceeds CF "{CF}" for {videoName}! Clearing. . .')
            frames=frames[:-CF]
        if len(frames) == CF:
            sumFrames = cv2.convertScaleAbs(sum(frames))
            contours, hierarchy = cv2.findContours(frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
            #cv2.drawContours(frame, contours, -1, (0, 255, 0), 2)  # Draw all contours in green color with thickness 2

        scaleBar(frame, scaleFactor=12.6, scaleLength=30, divisions = 6, thickness=thickness, fontScale=fontScale, barHeight = 20)
        fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);
        msgFPS = f'FPS : {str(int(fps))}'
        pos=(5, 5)
        rectFPS, posFPS = textBackground(msgFPS, fontScale, thickness, pos)
        cv2.rectangle(frame, rectFPS[0], rectFPS[1], color=(255, 255, 255), thickness=-1)
        cv2.putText(frame, msgFPS, posFPS, cv2.FONT_HERSHEY_SIMPLEX, fontScale, (0,0,0), thickness, lineType=cv2.LINE_AA)

        #sleep(1/cap.get(cv2.CAP_PROP_FPS)) 
        cv2.imshow(videoName, frame)
        if cv2.waitKey(100) & 0xFF == ord('q'):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
contours, hierarchy = cv2.findContours(sumFrames, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                for contour in contours: